In [315]:
import pandas as pd
import numpy as np
import datetime

## Read Data

In [349]:
df = pd.read_csv("V2_ENG_interview_dataset.csv", infer_datetime_format=True)
#df = df.dropna(axis=1)

/tmp/ipykernel_60960/2466963074.py:1: DtypeWarning: Columns (12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("V2_ENG_interview_dataset.csv", infer_datetime_format=True)


Drop bad rows

In [350]:
df.shape

(442458, 15)

In [351]:
df = df.loc[df["Unnamed: 10"].isna()]

In [352]:
df.shape

(442183, 15)

Test if are bad Lines


In [353]:
pd.to_numeric(df["Amount"], errors="raise");

In [354]:
df = df.dropna(axis=1)

In [355]:
df

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC)
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,890,NACAS,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,368.96,SHERATON HOTEL,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,SHERATON
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,165.82,SEARS.COM 9300,07/29/2013 12:00:00 AM,07/31/2013 12:00:00 AM,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...
3,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,96.39,WAL-MART #0137,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"GROCERY STORES,AND SUPERMARKETS"
4,201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,125.96,STAPLES DIRECT,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRIT..."
...,...,...,...,...,...,...,...,...,...,...
442453,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,992,TOTAL VALVE SYSTEMS,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,HARDWARE EQUIPMENT AND SUPPLIES
442454,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,2415,PRYOR WASTE & RECYCLING L,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,"UTILITIES - ELECTRIC,GAS,WATER,SANITARY"
442455,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,177.5,ACCURATE ENVIROMENTAL LAB,06/20/2014 12:00:00 AM,06/23/2014 12:00:00 AM,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED
442456,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,32457,RED BUD SERVICE INC,06/19/2014 12:00:00 AM,06/23/2014 12:00:00 AM,PROFESSIONAL SERVICES NOT ELSEWHERE CLASSIFIED


In [356]:
df["Posted Date"] = pd.to_datetime(df["Posted Date"])
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"])

In [357]:
df["Days Delay"] = (df["Posted Date"] - df["Transaction Date"]).dt.days

In [358]:
df

,Year-Month,Agency Number,Agency Name,Cardholder Last Name,Cardholder First Initial,Amount,Vendor,Transaction Date,Posted Date,Merchant Category Code (MCC),dif
0,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,890,NACAS,2013-07-30,2013-07-31,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS,1
1,201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,368.96,SHERATON HOTEL,2013-07-30,2013-07-31,SHERATON,1
2,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,165.82,SEARS.COM 9300,2013-07-29,2013-07-31,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...,2
3,201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,96.39,WAL-MART #0137,2013-07-30,2013-07-31,"GROCERY STORES,AND SUPERMARKETS",1
4,201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,125.96,STAPLES DIRECT,2013-07-30,2013-07-31,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRIT...",1
...,...,...,...,...,...,...,...,...,...,...,...
442453,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,992,TOTAL VALVE SYSTEMS,2014-06-20,2014-06-23,HARDWARE EQUIPMENT AND SUPPLIES,3
442454,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,2415,PRYOR WASTE & RECYCLING L,2014-06-20,2014-06-23,"UTILITIES - ELECTRIC,GAS,WATER,SANITARY",3
442455,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,177.5,ACCURATE ENVIROMENTAL LAB,2014-06-20,2014-06-23,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED,3
442456,201406,98000,GRAND RIVER DAM AUTH.,Cardholder,e,32457,RED BUD SERVICE INC,2014-06-19,2014-06-23,PROFESSIONAL SERVICES NOT ELSEWHERE CLASSIFIED,4
